The aim is to plot noisy classical simulation results.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
if "PLOT_MATERIAL" not in os.listdir():
  %mkdir "PLOT_MATERIAL"

Retrieve simulation results



In [2]:
dir = '../result/'
data_names = ['4_sites','6_sites','8_sites','10_sites','12_sites']
dir_names = ['0720_1434_okuda','0720_1357_okuda','0720_0850_okuda','0724_2034_okuda','0715_1644_okuda']
result_dirs = [dir+'/'+dir_names[0],dir+'/'+dir_names[1],dir+'/'+dir_names[2],dir+'/'+dir_names[3],dir+'/'+dir_names[4]]
for j in range(len(data_names)):
  if result_dirs[j] not in sys.path:
    sys.path.append(result_dirs[j])  

In [3]:
def is_number(st):
  try:
    float(st)
    return True
  except ValueError:
    return False
        
para_dic_list = [{} for each in data_names] 
for j in range(len(data_names)):
  print("j =",j)
  with open('{}/{}'.format(result_dirs[j],'config.txt'),'r') as f1:
    content = f1.readlines()
  with open('./PLOT_MATERIAL/config_{}.py'.format(data_names[j]),'w') as f2:
    for i in range(2):
      pass
    for i in range(len(content)-5):
      line_as_list = content[2+i].split()
      if line_as_list[1] == "=":
        s = ""
        for k in range(2,len(line_as_list)):
          s = s + line_as_list[k]
        para_dic_list[j].update({line_as_list[0]:s}) 
      else:
        s = ""
        for k in range(1,len(line_as_list)):
          s = s + line_as_list[k]
        para_dic_list[j].update({line_as_list[0]:s})
      f2.write(line_as_list[0]+" = ")
      for each in line_as_list[2:]:
        if is_number(each) or each[0]=='[' or each[-1]==']' or each=='True' or each=='False' or each[-1]==',':
          f2.write(each)
        else:
          f2.write('\'')
          f2.write(each)
          f2.write('\'')
        f2.write(' ')
      f2.write('\n')

j = 0
j = 1
j = 2
j = 3
j = 4


In [4]:
print("Different parameters:\n")
for each in para_dic_list[0].keys():
  equality = True
  for j in range(1,len(data_names)):
    if each not in para_dic_list[j].keys():
      equality = False
      para_dic_list[j].update({each:'None'})
    if para_dic_list[0][each] != para_dic_list[j][each]:
      equality = False
  if equality == False:
    print(each,": ",[para_dic_list[j][each] for j in range(len(data_names))])

Different parameters:

site :  ['4', '6', '8', '10', '12']
initial_layout :  ['[12,13,14,16]', '[12,13,14,16,19,22]', '[12,13,14,16,19,22,25,24]', '[12,13,14,16,19,22,25,24,23,21]', '[12,13,14,16,19,22,25,24,23,21,18,15]']


In [13]:
def pauliopexp(pauliop,ini_st):
  exp = 1
  for position in range(len(pauliop)):
    if pauliop[position] == ini_st[position][1]:
        exp *= (-1)**ini_st[position][0]
    if pauliop[position] == "I": 
        continue
    if pauliop[position] != "I" and pauliop[position] != ini_st[position][1]:
        exp = 0
        break
  return exp 

In [14]:
if './SCRIPT' not in sys.path: sys.path.append('./SCRIPT')
obs_label_dic = {'charge':'Q','char_dif':'Q_dif'}
ans_lists = [[] for each in data_names]
ans_exact_list = []
stat_unc_lists = [[] for each in data_names]
for j in range(len(data_names)):
  print("j =",j)
  g = open('../result/.current_result_dir.txt', 'w')
  g.write('{}'.format(result_dirs[j]))
  g.close()
  %cp './PLOT_MATERIAL/config_{data_names[j]}.py' './GENERATED/parameters.py'
  ! python3 ./SCRIPT/get_coef.py
  with open('./GENERATED/coefficients.dat') as f:
    original = f.readlines()
  
  if para_dic_list[j]['initial_state'] == 'Neel':
        ini_st = [[(k+1)%2,'Z'] for k in range(int(para_dic_list[j]['site']))]
  if para_dic_list[j]['initial_state'] == 'file':  
        with open('{}/initial_state.txt'.format(result_dirs[j]),'r') as g:
            bit_string = g.readline().split()
            pauli_string = g.readline().split()
        ini_st = [ [ int(bit_string[k]), pauli_string[k] ] for k in range(int(para_dic_list[j]['site'])) ]

  for i in range(int(para_dic_list[0]['step_initial']),int(para_dic_list[j]['trotter_step'])+1):
    ans = 0
    g = open('{}/LOCAL_OBS/local_obs_res_{}.txt'.format(result_dirs[j],i), 'r')
    result = g.readlines()
    g.close()
    ans_exact = 0
    for k in range(len(original)):
        degree, coeff, pauliop = original[k].split()
        res = float(result[k])
        degree = int(degree)
        coeff = int(coeff)
        ans += res * (float(para_dic_list[j]['delta']) ** degree) * coeff
        ans_exact += pauliopexp(pauliop,ini_st) * (float(para_dic_list[j]['delta']) ** degree) * coeff
    ans_lists[j].append(ans)
    if i == 0:
      ans_exact_list.append(ans_exact)

print("ans_exact_list =",ans_exact_list)  

j = 0
sys.path[0]= /Users/takuya/Documents/GitHub/integrable-trotterization/public_code/SCRIPT
j = 1
sys.path[0]= /Users/takuya/Documents/GitHub/integrable-trotterization/public_code/SCRIPT
j = 2
sys.path[0]= /Users/takuya/Documents/GitHub/integrable-trotterization/public_code/SCRIPT
j = 3
sys.path[0]= /Users/takuya/Documents/GitHub/integrable-trotterization/public_code/SCRIPT
j = 4
sys.path[0]= /Users/takuya/Documents/GitHub/integrable-trotterization/public_code/SCRIPT
ans_exact_list = [-2.0, -6.0, -6.0, -8.0, -12.0]


In [ ]:
#%%script false 
for j in range(len(data_names)):
  print("j =",j)
  print(ans_lists[j])

: 

Fit and plot

In [ ]:
import scipy.optimize
from scipy.optimize import curve_fit,leastsq

###  Adjust colors, labels, markers, and ylabel according to your choice
colors = ['tab:blue','tab:orange','tab:green','tab:red','tab:purple']
cs = [1.5,1.1,2,2,1.6]
labels =  [f'4 sites, $C={cs[0]}$',f'6 sites, $C={cs[1]}$',f'8 sites, $C={cs[2]}$',f'10 sites, $C={cs[3]}$',f'12 sites, $C={cs[4]}$']
markers = ["o","s","^","v","*"]
ylabel = r"$ \langle Q_1^{\rm dif}\rangle/C$" 
sign_overall = +1

steps =  range(int(para_dic_list[0]['step_initial']),int(para_dic_list[0]['trotter_step'])+1)
print("steps =",steps)
def exp_func(step, v0, rate, shift):
  return v0 * np.exp(-rate * step) + shift

def sign_st(value):
  if np.sign(value) == +1: return "+"
  if np.sign(value) == -1: return "-"
  else: return "0"

for j in range(0,len(data_names)):
  print("j =",j)
  vals = sign_overall* np.array(ans_lists[j])
  print("len(vals) =",len(vals))
  unc_array = sign_overall*np.array(stat_unc_lists[j])

  ### Plot
  plt.plot(steps[:10],vals[:10]/cs[j],,color=colors[j], marker=markers[j],label=labels[j],linestyle="none")

  ### Draw a horizontal line

  plt.axhline(y = ans_exact_list[j]/cs[j], color = colors[j], linestyle = ':',linewidth=1)
  print("ans_exact_list[j] =",ans_exact_list[j])
  print("ans_exact_list[j]/cs[j] =",ans_exact_list[j]/cs[j])


plt.axhline(y = 0, color = 'black' , linestyle = 'dashed',linewidth=1)
plt.xlabel(r"Trotter step $d$")
plt.ylabel(ylabel)

file_name = ''
for j in range(len(data_names)):
  file_name += dir_names[j]
  file_name += "_"

plt.legend(loc = (0.58,0.02))
fig = plt.gcf()
fig.set_size_inches(0.8*6.4,0.8*4.8)
file_name_full = '{}/{}fitted.pdf'.format('./PLOT_MATERIAL',file_name)
print(file_name_full)
plt.savefig(file_name_full)
!date 

: 

: 